# Figure 4: Distance permutation analysis 

In [1]:
import rpy2

In [2]:
%reload_ext rpy2.ipython

: 

: 

In [ ]:
%%R -i cells_df2

# load packages
library(tidyverse)
library(here)
source("triangulation_distances.R")

# Define columns of input data 
compare_condition_column <- "day harvested"
cell_index_column <- "index"
x_position_column <- "x"
y_position_column <- "y"
cell_type_column <- "Cell common"
region_column <- "unique_region"


# Define file path to save avg. distances 
filepath_avg_dist <- "Results"
dir.create(filepath_avg_dist)

# Define number of iterations for iterated distances
number_of_iterations <- 100
distance_threshold = 128
# Settings for Dumbbell plot
#pairs_for_comparisson_Dumbbell_plot <- c(unique(df_full$cell_type))

#################################################################################

df= cells_df2

colnames(df)[1] = "index"
cell_index_column <- "index"
df = df[,c(cell_index_column, x_position_column, y_position_column, compare_condition_column, region_column, cell_type_column)] 

metadata <- df %>%
  dplyr::select(compare_condition_column, region_column) %>%
  dplyr::distinct(.keep_all = TRUE)    

#################################################################################


triangulation_distances <- get_triangulation_distances(df_input = df,
                                                       id = cell_index_column,
                                                       x_pos = x_position_column,
                                                       y_pos = y_position_column,
                                                       cell_type = cell_type_column,
                                                       region = region_column,
                                                       calc_avg_distance = TRUE,
                                                       csv_output = filepath_avg_dist,
                                                       num_cores = 16)

head(triangulation_distances)

write.csv(triangulation_distances, paste0(filepath_avg_dist, "/", "triangulation_distances", ".csv"))

# Iterations
# In the iterated distances, distances are summarized per region for each iteration.
#Note: you don't need to shuffle the cell annotations yourself, it's done in the iteration for you



iterated_triangulation_distances <- iterate_triangulation_distances(df_input = df,
                                                                    id = cell_index_column,
                                                                    x_pos = x_position_column,
                                                                    y_pos = y_position_column,
                                                                    cell_type = cell_type_column,
                                                                    region = region_column,
                                                                    num_iterations = number_of_iterations,
                                                                    num_cores = 16)
#head(iterated_triangulation_distances)

write.csv(iterated_triangulation_distances, paste0(filepath_avg_dist, "/", "iterated_triangulation_distances","_",as.character(number_of_iterations),  ".csv"))  


In [ ]:
%%R

#### Further analysis triangulation - modified
names(metadata)[names(metadata) == compare_condition_column] <- "treatment"
# Reformat observed dataset
observed_distances <- triangulation_distances %>%
  # Append metadata
  dplyr::left_join(metadata,
    by = c("unique_region")
  ) %>%
  dplyr::filter(distance <= distance_threshold) %>%
  # Calculate the average distance to every cell type for each cell
  dplyr::group_by(celltype1_index, celltype1, celltype2, treatment, unique_region) %>%
  dplyr::summarize(mean_per_cell = mean(distance)) %>%
  dplyr::ungroup() %>%
  # Calculate the average distance between cell type to cell type on a per group basis
  dplyr::group_by(celltype1, celltype2, treatment) %>%
  dplyr::summarize(
    observed = list(mean_per_cell),
    observed_mean = mean(unlist(observed), na.rm = TRUE)
  ) %>%
  dplyr::ungroup()

# Reformat exepcted dataset
expected_distances <- iterated_triangulation_distances %>%
  # Append metadata
  dplyr::left_join(metadata,
    by = c("unique_region")
  ) %>%
  dplyr::filter(mean_dist <= distance_threshold) %>%
  # Calculate expected mean distance and list values
  dplyr::group_by(celltype1, celltype2, treatment) %>%
  dplyr::summarize(
    expected = list(mean_dist),
    expected_mean = mean(mean_dist, na.rm = TRUE)
  ) %>%
  dplyr::ungroup()

# drop comparisons with low numbers of observations 
# This step was implemented to reduce the influence of rare cell types - usually, these tend to dominate statistics as small changes are already highly significant 
logical_list_observed <- list()
for (i in 1:nrow(observed_distances)) {
  print(i)
  vec <- observed_distances[i, "observed"]

  if (length(unlist(vec)) > 10) {
    logical_list_observed[[as.character(i)]] <- TRUE
  } else {
    logical_list_observed[[as.character(i)]] <- FALSE
  }
}

list_observed <- unlist(logical_list_observed)  

observed_distances$keep <- list_observed

observed_distances <- observed_distances %>% filter(keep == TRUE)

# Perform the same filtering on the expected distances
# drop comparisons with low numbers of observations 
# This step was implemented to reduce the influence of rare cell types - usually, these tend to dominate statistics as small changes are already highly significant 
logical_list_expected <- list()
for (i in 1:nrow(expected_distances)) {
  print(i)
  vec <- expected_distances[i, "expected"]

  if (length(unlist(vec)) > 10) {
    logical_list_expected[[as.character(i)]] <- TRUE
  } else {
    logical_list_expected[[as.character(i)]] <- FALSE
  }
}

list_observed <- unlist(logical_list_expected)  

expected_distances$keep <- list_observed

expected_distances <- expected_distances %>% filter(keep == TRUE)

# Calculate pvalues and log fold differences
distance_pvals <- expected_distances %>%
  dplyr::left_join(observed_distances,
    by = c("celltype1", "celltype2", "treatment")
  ) %>%
  # Calculate wilcoxon test between observed and expected distances
  dplyr::group_by(celltype1, celltype2, treatment) %>%
  dplyr::mutate(pvalue = wilcox.test(unlist(expected), unlist(observed), exact = FALSE)$p.value) %>%
  dplyr::ungroup() %>%
  dplyr::select(-observed, -expected) %>%
  # Calculate log fold enrichment
  dplyr::mutate(
    logfold_group = log2(observed_mean / expected_mean),
    interaction = paste0(celltype1, " --> ", celltype2)
  )

# Get order of plot by magnitude of logfold differences between groups
intermed <- distance_pvals %>%
  dplyr::select(interaction, treatment, logfold_group) %>%
  tidyr::spread(key = treatment, value = logfold_group)

#intermed$difference <- (intermed[, condition_2] - intermed[, condition_1])

#ord <- (intermed %>%
 # dplyr::filter(!is.na(difference)) %>%
  #dplyr::arrange(condition_1))$interaction

# Assign interaction order
#distance_pvals$interaction <- factor(distance_pvals$interaction,
 # levels = ord
#)

distance_pvals <- write_csv(distance_pvals, paste0(specific_output, "/distance_pvals.csv"))

# general filtering before analysis of the results
distance_pvals_sig <- distance_pvals %>%
  filter(pvalue < 0.05) %>% # keep only significant results
  filter(celltype1 != celltype2) %>% # compare only different cell types 
  filter(!is.na(observed_mean)) %>% # ignore columns without observation
#  filter(celltype1 != "unknown") %>% # drop cells of type unknown
 # filter(celltype2 != "unknown") %>%
#  filter(celltype1 != "noise") %>% # drop cells of type noise 
 # filter(celltype2 != "noise") %>% 
  filter(!is.na(treatment))
  
#################################################################################

distance_pvals_sig <- distance_pvals_sig %>% 
  filter(!is.na(interaction)) %>% 
  filter(!is.na(logfold_group))
distance_pvals_sig$abs_logfold <- abs(distance_pvals_sig$logfold_group)
distance_pvals_sig_filt <- distance_pvals_sig %>% filter(abs_logfold >= 0.5)
distance_pvals_sig_filt <- distance_pvals_sig_filt[!duplicated(t(apply(distance_pvals_sig_filt[c("celltype1", "celltype2")], 1, sort))), ]

distance_pvals_interesting <- distance_pvals[distance_pvals$interaction %in% distance_pvals_sig_filt$interaction, ]

distance_pvals_interesting <- distance_pvals_interesting %>% filter(!is.na(treatment))

distance_pvals_interesting <- distance_pvals[distance_pvals$interaction %in% comb_filt$interaction, ]
distance_pvals_interesting <- distance_pvals_interesting %>% filter(!is.na(treatment))

#################################################################################

# Pairs to plot in Dumbell plot
pair_to = unique(distance_pvals_sig_filt$interaction)

# Colors used in Dumbell plot 
colors = c("#8de4d3", "#a21636", "#94ea5b")

#################################################################################

# Dumbbell plot
data <- distance_pvals %>%
  dplyr::filter(!is.na(interaction))

distance_pvals$pairs <- paste0(distance_pvals$celltype1, "_", distance_pvals$celltype2)
distance_pvals_sub <- distance_pvals[distance_pvals$interaction %in% pair_to, ]

distance_pvals_sub <- distance_pvals_sub %>% filter(!is.na(treatment)) %>% arrange(logfold_group) %>% mutate(interaction = factor(interaction, unique(interaction)))
#distance_pvals_sub$interaction <- factor(distance_pvals_sub$interaction, levels=unique(distance_pvals_sub$interaction))


distance_pvals_sub_filt <- distance_pvals_sub #%>% filter(!treatment == 5)
distance_pvals_sub_filt$treatment <- as.factor(distance_pvals_sub_filt$treatment)

ggplot2::ggplot(data = distance_pvals_sub_filt %>%
  dplyr::filter(!is.na(interaction))) +
  ggplot2::geom_vline(mapping = ggplot2::aes(xintercept = 0), linetype = "dashed") +
  ggplot2::geom_line(
    mapping = ggplot2::aes(x = logfold_group, y = interaction),
    na.rm = TRUE
  ) +
  ggplot2::geom_point(
    mapping = aes(x = logfold_group, y = interaction, fill = treatment, shape = treatment),
    size = 4, stroke = 0.5, na.rm = TRUE
  ) +
  ggplot2::scale_shape_manual(values = c(24, 22, 23)) +
  ggplot2::scale_fill_manual(values = colors) +
  ggplot2::theme_bw() +
  ggplot2::theme(
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    axis.text.y = element_text(size = 16),
    axis.text.x = element_text(size = 16, angle = 45, hjust = 1),
    axis.title.y = element_text(size = 16),
    axis.title.x = element_text(size = 16)
  )

ggsave(paste0(output_path, "dumbbell.pdf"))

#################################################################################

library(tidyverse)
library(igraph)
library(ggraph)
library(scales)
library(ggplot2)

pairs <- unique(distance_pvals_sig_filt$interaction)

distance_pvals_sub2 <- distance_pvals[distance_pvals$interaction %in% pairs, ]


distance_pvals_sub_grouped <- distance_pvals_sub2 %>% group_by(celltype1, celltype2)
distance_pvals_sub_grouped <- na.omit(distance_pvals_sub_grouped)

distance_pvals_sub2 <- na.omit(distance_pvals_sub2)

pairs <- unique(distance_pvals_sub2$interaction)

result_list_day1 <- list()
result_list_day3 <- list()
result_list_day5 <- list()

for (p in pairs) {
  distance_pvals_sub_filt_1 <- distance_pvals_sub2 %>%
    filter(interaction == p) %>%
    filter(treatment == 1)
  distance_pvals_sub_filt_2 <- distance_pvals_sub2 %>%
    filter(interaction == p) %>%
    filter(treatment == 3)
  distance_pvals_sub_filt_3 <- distance_pvals_sub2 %>%
    filter(interaction == p) %>%
    filter(treatment == 5)

  if (nrow(distance_pvals_sub_filt_1) > 0) {
    if (0 > distance_pvals_sub_filt_1$logfold_group) {
      direction <- "#3976AC"
    } else {
      direction <- "#C63D30"
    }

    df_res2 <- data.frame(
      celltype1 = distance_pvals_sub_filt_1$celltype1,
      celltype2 = distance_pvals_sub_filt_1$celltype2,
      logfold = distance_pvals_sub_filt_1$logfold_group,
      direction = direction
    )
    result_list_day1[[p]] <- df_res2
  }

  if (nrow(distance_pvals_sub_filt_2) > 0) {
    if (0 > distance_pvals_sub_filt_2$logfold_group) {
      direction <- "#3976AC"
    } else {
      direction <- "#C63D30"
    }

    df_res1 <- data.frame(
      celltype1 = distance_pvals_sub_filt_2$celltype1,
      celltype2 = distance_pvals_sub_filt_2$celltype2,
      logfold = distance_pvals_sub_filt_2$logfold_group,
      direction = direction
    )
    result_list_day3[[p]] <- df_res1
  }

  if (nrow(distance_pvals_sub_filt_3) > 0) {
    if (0 > distance_pvals_sub_filt_3$logfold_group) {
      direction <- "#3976AC"
    } else {
      direction <- "#C63D30"
    }

    df_res3 <- data.frame(
      celltype1 = distance_pvals_sub_filt_3$celltype1,
      celltype2 = distance_pvals_sub_filt_3$celltype2,
      logfold = distance_pvals_sub_filt_3$logfold_group,
      direction = direction
    )
    result_list_day5[[p]] <- df_res3
  }
}

graph_df_day1 <- as.data.frame(do.call(rbind, result_list_day1))
graph_df_day3 <- as.data.frame(do.call(rbind, result_list_day3))
graph_df_day5 <- as.data.frame(do.call(rbind, result_list_day5))

graph_list <- list(graph_df_day1, graph_df_day3, graph_df_day5)

for (x in 1:3) {
  graph_df <- graph_list[[x]]


  graph_df <- graph_df[!duplicated(t(apply(graph_df[c("celltype1", "celltype2")], 1, sort))), ]

  mat <- graph_df

  cci_control <- mat


  g <- graph_from_data_frame(data.frame(cci_control))
  E(g)$weights <- ifelse(cci_control$logfold == 0,
    1e-10, abs(cci_control$logfold)
  )

  color_dic_cells <- c('Tumor PDL1+ MHCI+' = "#21f0b6",
                     'CD8+ T cell PD1+' = "#2a8665",
                     'Tumor TYRP1+' = "#79c6c1",
                     'DC' = "#2a538a",
                     'Macrophage' = "#daa4f9",
                     'Tumor' = "#7e39c2",
                     'Endothelial CD106+' = "#9ab9f9",
                     'CD8+ T cell' = "#9e3678",
                     'DC TCF7+' = "#f36ad5",
                     'Epiehtlial' = "#9f04fc",
                     'CD4+ T cell' = "#5ac230",
                     'Epithelial' = "#b7d165",
                     'Tumor Ki67+' = "#6d3918",
                     'CD4+ Treg' = "#efaa79",
                     'Neutrophil' = "#9f2114",
                     'Endothelial' = "#fd5917",
                     'NK' = "#fe1d66",
                     'Macrophage PDL1+' = "#f7767d",
                     'APC MHCIIhi' = "#fbbd13",
                     'Macrophage CD86+' = "#748d13",
                     'Lymphatic' = '#00944F',
                     'CD86+ Macrophage' = '#636A86',
                     'Macrophage CD169+' = '#7A50F7',
                     'Tumor CD117hi' = '#24B93B',
                     'Lymphatic Ly6Chi' = '#9BF29D',
                     'NK cell KLRG1hi' = '#829824',
                     'B cell' = '#F2A3F0')

  
  
  V(g)$color <- color_dic_cells[unique(c(mat$celltype1, mat$celltype2))] 
  # pdf("figures/WilkEtAl/cellchat_CCI_network_byCondition_Control_network.pdf",
  #     width = 8,
  #     height = 6)


  radian.rescale <- function(x, start = 0, direction = 1) {
    c.rotate <- function(x) (x + start) %% (2 * pi) * direction
    c.rotate(scales::rescale(x, c(0, 2 * pi), range(x)))
  }

  lab.locs <- radian.rescale(x = 1:18, direction = -1, start = 0)

  plot(g,
    vertex.size = 17,
    vertex.color = V(g)$color,
    vertex.label.color = "black",
    vertex.label.cex = 1,
    #vertex.label.dist = 2.5,
    vertex.label.degree = lab.locs,
    edge.width = E(g)$weights * 15,
    edge.arrow.size = log(1 / E(g)$weights) / 80,
    edge.color = E(g)$direction,
    edge.curved = 0,
    asp = 0.9,
    layout = layout_in_circle,
    main = paste0(compare_condition_column, "_", unique(graph_df$group))
  )
  # dev.off()
}


##############################################################
### CN interface detection (Step 14)

# Spatial Context
data = cells_df2
col_list = data.columns
# Spatial context 
n_num = 75
ks=[n_num]
cluster_col = "Neighborhood common"
sum_cols=cells_df2[cluster_col].unique()
keep_cols = col_list
X='x'
Y='y'
#Neigh = Neighborhoods(cells_df2,ks,cluster_col,sum_cols,keep_cols,X,Y,reg=Reg,add_dummies=True)
#windows = Neigh.k_windows()
reg = "unique_region"

windows, sum_cols = tl_Create_neighborhoods(df = data,
                     n_num = n_num,
                     cluster_col = cluster_col,
                     X = X,
                     Y = Y,
                     regions = reg,
                     sum_cols = None,
                     keep_cols = None,
                     ks = [n_num])

w, k_centroids = tl_Chose_window_size(windows,
                      n_num = n_num,
                      n_neighborhoods = 10,
                      n2_name = 'neigh_ofneigh', sum_cols = sum_cols)

pl_Niche_heatmap(k_centroids, w, n_num, sum_cols)


names = cells_df2[cluster_col].unique()
colors = hf_generate_random_colors(n = len(names))

color_dic = hf_assign_colors(names, colors)

color_dic=color_dic
l=list(color_dic.keys())



plot_list = [ 'Vasculature' , 'Tumor','Immune Infiltrate']

pl_Barycentric_coordinate_projection(w, 
                                      plot_list = plot_list, 
                                      threshold = 70, 
                                      output_dir = output_dir, 
                                      save_name = save_name, 
                                      col_dic = color_dic,
                                      l = l,
                                      cluster_col = cluster_col,
                                      n_num = n_num,
                                      SMALL_SIZE = 14, 
                                      MEDIUM_SIZE = 16, 
                                      BIGGER_SIZE = 18,
                                      figsize=(6,5))


##############################################################
### CN spatial context analysis (Step 15)

simps, simp_freqs, simp_sums = tl_calculate_neigh_combs(w, 
                                                     l,
                                                     n_num, 
                                                     threshold = 0.85, 
                                                     per_keep_thres = 0.85)

g, tops, e0, e1 = tl_build_graph_CN_comb_map(simp_freqs)

pl_generate_CN_comb_map(graph = g, 
                     tops = tops, 
                     e0 = e0, 
                     e1 = e1, 
                     l = l,
                     simp_freqs = simp_freqs,
                     color_dic = color_dic)

pl_get_network(ttl_per_thres=0.9,
            comb_per_thres=0.005,
            neigh_sub = plot_list,
            save_name='All_comm',
            save_path = output_dir,
            sub_col = cluster_col, 
            color_dic = color_dic,
            windows = windows,
            n_num = n_num,
            l = l)


##############################################################
### Patch-based fragmentation analysis (Step 16)

data_path = "Z:/admin/Tim/Fragmentation_analysis_pub/Data/HuBMAP_gut/CODEX_HuBMAP_alldata_Dryad.csv"
voronoi_output_path = "Z:/admin/Tim/Fragmentation_analysis_pub/Mock_test_set/Results/Voronoi/"

df = pd.read_csv(data_path)
generate_voronoi_plots(df, voronoi_output_path)

image_folder = 'D:/Tim/doi_10.5061_dryad.76hdr7t1p__v3'
mask_output = "Z:/admin/Tim/Fragmentation_analysis_pub/Mock_test_set/Results/Masks/"

generate_masks_from_images(image_folder, mask_output, image_type = ".tif", filter_size = 5, threshold_value = 10)

filter_list = ['colon', 'B009Bt', 'SB']

df_info = generate_info_dataframe(df, voronoi_output_path, mask_output, filter_list)

folder_names = filtered_df['folder_names'].unique()
print(filtered_df)
print(folder_names)

voronoi_path, mask_path, region = hf_process_dataframe(df_info)

process_files(voronoi_path, mask_path, region)
    
df_info = df_info
output_dir_csv ="C:/Users/Tim/Downloads/Test_patch_20230712/csv"

results_df, contour_list = process_data(df_info, output_dir_csv)